## Выполненное задание по практике раздела 3.1.


- Для выполнения задания использовалось следующее:
-- wsl2 (Ubuntu 18.04.5 LTS)
-- docker compose
-- сборка Hadoop https://github.com/m-semnani/bd-infra.git (явно не запустился HUE - конфликт с MySQL, далее надо попробовать предлагаемый docker образ на LMS)


- Ниже указаны задания и код решения

### 1. Когда мы перетащили файлы с произведением Льва Толстого – мы перетащили их в файловую систему виртуальной машины, но не в HDFS, соответственно, в первую очередь нам нужно перенести их в папку нашего пользователя именно на HDFS.
    
    ```
        /* Объединение файлов в обычной косоли  */
        cat *.txt > vm.txt

        /* Копия файлов в контейнер докера  */
        docker cp vm.txt 5a3c4c54ea92:/home/books

        /* Копия файлов из локальной папки в распределенную систему HDFS
        с предварительным созданием каталога */
        hadoop fs -mkdir /db/tolstoy
        hadoop fs -put -f /home/books/*.txt /db/tolstoy
    ```
    
### 2. После того, как файлы окажутся на HDFS попробуйте выполнить команду, которая выводит содержимое папки. Особенно обратите внимание на права доступа к вашим файлам.
    
    ```
        /* Содержимое папки */
        hadoop fs -ls /db/tolstoy/
        >> -rw-r-r   2 root supergroup    3048008 2022-11-15 10:48 /db/tolstoy/vm.txt
    ```
    
### 3. Далее сожмите все 4 тома в 1 файл.
      
    ```
        /* Сдедад ранее в обычной консоли  */
        cat *.txt > vm.txt
    ```
    
### 4. Теперь давайте изменим права доступа к нашему файлу. Чтобы с нашим файлом могли взаимодействовать коллеги, установите режим доступа, который дает полный доступ для владельца файла, а для сторонних пользователей возможность читать и выполнять.
    
    ```
        /* */
        hadoop fs -chmod u+x ./db/tolstoy/vm.txt
        hadoop fs -chmod go-rwx ./db/tolstoy/vm.txt
        hadoop fs -chmod go+rx ./db/tolstoy/vm.txt

    ```
    
### 5. Попробуйте заново использовать команду для вывода содержимого папки и обратите внимание как изменились права доступа к файлу.

    ```
        >> -rwx-rx-rx   2 root supergroup    3048008 2022-11-15 10:48 /db/tolstoy/vm.txt
    ```
### Теперь попробуем вывести на экран информацию о том, сколько места на диске занимает наш файл. Желательно, чтобы размер файла был удобночитаемым.
    
    ```    
        /* Вывод размера файла */
        root@5a3c4c54ea92:/home/books# hdfs dfs -du -h /db/tolstoy/vm.txt
        >> 2.9 M  /db/tolstoy/vm.txt

        /* Вывод статистики по файлу */
        root@5a3c4c54ea92:/home/books# hadoop fs -stat "file_size:%b  replication:%r name:%n" /db/tolstoy/vm.txt
        >> file_size:3048008  replication: name:vm.txt
    ```
    
### 6. На экране вы можете заметить 2 числа. Первое число – это фактический размер файла, а второе – это занимаемое файлом место на диске с учетом репликации. По умолчанию в данной версии HDFS эти числа будут одинаковы – это означает, что никакой репликации нет – нас это не очень устраивает, мы хотели бы, чтобы у наших файлов существовали резервные копии, поэтому напишите команду, которая изменит фактор репликации на 2.
    
    ```
        /* Изменение числа репликации файла */
        hdfs dfs –setrep 2 /user/hdfs/file.txt

    ```

### 7. Повторите команду, которая выводит информацию о том, какое место на диске занимает файл и убедитесь, что изменения произошли.

    ```
        root@5a3c4c54ea92:/home/books# hadoop fs -stat "file_size:%b block_size:%o replication:%r name:%n" /db/tolstoy/vm.txt
        >> file_size:3048008 block_size:134217728 replication:2 name:vm.txt
    ```
        
### 8. Напишите команду, которая подсчитывает количество строк в вашем файле
        
        ```
            hadoop fs -cat hadoop fs -cat /db/tolstoy/vm.txt | wc -l
            >> 10272
        ```
        
 